# FAF.FIA16.1 -- Artificial Intelligence Fundamentals

> **Lab 4:** Processing Images with OpenCV \\
> **Performed by:** Cambur Dumitru, group FAF-191 \\
> **Verified by:** Mihail Gavrilita





## Imports and Utils

In [3]:
import cv2
import matplotlib.pyplot as plt


## Task 1 -- Write the following functions using OpenCV. Adjust the parameters and explain your approach. Plot the initial image and the blurred image in the same plot by using Matplotlib subplots:

In [4]:
def blur_image(img, kernel_size=5):
    blurred = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    return blurred


def sharpen_image(img, kernel_size=5, amount=1, threshold=0):
    blurred = cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)
    sharpened = cv2.addWeighted(img, 1 + amount, blurred, -amount, threshold)
    return sharpened


img = cv2.imread('test_images/0AA0A2.jpg')
blurred_img = blur_image(img)
sharpened_img = sharpen_image(img)

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax[0].set_title('Original Image')
ax[1].imshow(cv2.cvtColor(blurred_img, cv2.COLOR_BGR2RGB))
ax[1].set_title('Blurred Image')
plt.show()

fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
ax[0].set_title('Original Image')
ax[1].imshow(cv2.cvtColor(sharpened_img, cv2.COLOR_BGR2RGB))
ax[1].set_title('Sharpened Image')
plt.show()


[5, 9] [-1, -1] [6, 12] [4.0, 8.0] 26 -2


In the code above, for the sharpening algorithm "unsharp masking" is applied. Its name derives from the fact that the technique uses a blurred, or "unsharp", negative image to create a mask of the original image. The unsharp mask is then combined with the original positive image, creating an image that is less blurry than the original. The resulting image, although clearer, may be a less accurate representation of the image's subject. To control the unsharp masking typically amount, radius and threshold is used:

    Amount is listed as a percentage and controls the magnitude of each overshoot (how much darker and how much lighter the edge borders become). This can also be thought of as how much contrast is added at the edges. It does not affect the width of the edge rims.
    
    Radius affects the size of the edges to be enhanced or how wide the edge rims become, so a smaller radius enhances smaller-scale detail. Higher radius values can cause halos at the edges, a detectable faint light rim around objects. Fine detail needs a smaller radius. Radius and amount interact; reducing one allows more of the other.
    
    Threshold controls the minimal brightness change that will be sharpened or how far apart adjacent tonal values have to be before the filter does anything. This lack of action is important to prevent smooth areas from becoming speckled. The threshold setting can be used to sharpen more pronounced edges, while leaving subtler edges untouched. Low values should sharpen more because fewer areas are excluded. Higher threshold values exclude areas of lower contrast.

## Task 2 -- Implement a face detection system using OpenCV. The function should take as input one image and output the result as the coordinates of the face, in case the image contains a face, or None if the image does not contain any faces. Assume that the image contains no more than one face.:

In [5]:
def detect_face(image):
    # Load the pre-trained face detection classifier
    face_cascade = cv2.CascadeClassifier(
        cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    # Check if a face is detected
    if len(faces) > 0:
        # Assume that the image contains only one face
        # Return the coordinates of the detected face
        (x, y, w, h) = faces[0]
        #cv2.rectangle(img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        #cv2.imshow("img", img)
        #cv2.waitKey()
        return x, y, x + w, y + h
    else:
        # If no faces are detected, return None
        return None


img = cv2.imread('test_images/0AA0A2.jpg')
face = detect_face(img)


## Task 3 -- Task 3 Implement a system that detects if a photo is accepted for passport or not, by using OpenCV. You can be creative in determining the optimal strategy, but the system should at least follow the listed requirements.

In [6]:
def is_passport_photo(image):
    # Check if the image is colored
    if len(image.shape) < 3 or image.shape[2] < 3:
        return False

    # Check if the image is in portrait or square orientation
    (height, width, _) = image.shape
    if height > width:
        image = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        (height, width, _) = image.shape
    if height != width:
        return False

    # Detect the eyes in the image
    eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    eyes = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    if len(eyes) != 2:
        return False

    # Check if the eyes are at the same level
    (eye1_x, eye1_y, eye1_w, eye1_h) = eyes[0]
    (eye2_x, eye2_y, eye2_w, eye2_h) = eyes[1]
    eye1_center = eye1_y + (eye1_h / 2)
    eye2_center = eye2_y + (eye2_h / 2)
    if abs(eye1_center - eye2_center) > 5:
        return False

    # Check if the photo contains only one person
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    if len(faces) != 1:
        return False

    # Check if the head of the person represents 20% to 50% of the area of the photo
    (x, y, w, h) = faces[0]
    face_area = w * h
    image_area = width * height
    if face_area < (0.2 * image_area) or face_area > (0.5 * image_area):
        return False

    # If all checks pass, the photo is accepted
    return True


The main algorithm used to detect different features in an image is haar cascade. Haar cascade uses the cascading window, and it tries to compute features in every window and classify whether it could be an object. One can divide the algorithm into 4 steps:

1)Haar Features calculation (represented by taking a rectangular part of an image and dividing that rectangle into multiple parts)

2)Creation of Integral Images (image is where each pixel represents the cumulative sum of a corresponding input pixel with all pixels above and left of the input pixel. Speeds up the first step)

3)Using Adaboost (essentially chooses the best features and trains the classifiers to use them. It uses a combination of “weak classifiers” to create a “strong classifier” that the algorithm can use to detect objects)

4)Implementing Cascading Classifier itself.


## Task 4 --  Test how good your system performs on a test dataset. You are required to apply your system to all the images in the test set, then compute the accuracy for the solution.

In [8]:
import csv
from task3 import is_passport_photo
from task1 import blur_image, sharpen_image
test_folder = "test_images/"
test_csv_file = "test.csv"

# Read the labels from the test.csv file
with open(test_csv_file, "r") as f:
    reader = csv.DictReader(f)
    labels = {row['new_path']: row['label'] == 'True' for row in reader}

# Test the passport photo validator on each image in the test set
correct = 0
total = 0
for image_path, label in labels.items():
    image = cv2.imread(image_path)
    blur = blur_image(image)
    sharp = sharpen_image(blur)

    if is_passport_photo(sharp) == label:
        correct += 1
    total += 1

accuracy = correct / total
print(f"Accuracy: {accuracy:.2%}")


## Conclusions:

After implementing this laboratory work I've learnt about various image processing algorithms like blurring (smoothening), sharpening, haarcascade model innerworks and implemented all of this into a single project which identifies if image follows a required set of requirements to be applied into a passport

## Bibliography:

https://en.wikipedia.org/wiki/Unsharp_masking

https://www.analyticsvidhya.com/blog/2022/04/object-detection-using-haar-cascade-opencv/

https://medium.com/analytics-vidhya/haar-cascades-explained-38210e57970d